# 00. Setup and Build Environment

This notebook prepares the environment for all subsequent analysis. It handles:
0. **System Information**: Records hardware/software versions for reproducibility.
1. Checking for a GPU.
2. Cloning the project repository.
3. Installing Python dependencies.
4. Configuring the environment for compilation (detecting GPU architecture, setting OpenMP variables).
5. Building all C++/CUDA executables.

**Run this notebook once per session before running any of the other notebooks.**

## 0. System Information

In [ ]:
import platform
print(f"Python Version: {platform.python_version()}")
!nvidia-smi
!nvcc --version || echo 'nvcc not found'

## 1. GPU Check

In [ ]:
!nvidia-smi || echo "No GPU visible"

## 2. Clone Repository & Install Dependencies

In [ ]:
!git clone https://github.com/UchihaIthachi/sssp-apsp-hpc-openmp-cuda.git
%cd sssp-apsp-hpc-openmp-cuda
!git rev-parse HEAD
%pip install -q pandas==2.0.3 matplotlib==3.7.1 seaborn==0.12.2

## 3. Configure Build Environment

In [ ]:
import os, shutil, subprocess, multiprocessing
def detect_sm():
    try:
        name = subprocess.check_output(
            "nvidia-smi --query-gpu=name --format=csv,noheader",
            shell=True, text=True).strip()
        if "T4" in name: return "sm_75"
        if "V100" in name: return "sm_70"
        if "A100" in name: return "sm_80"
        if "L4" in name or "4090" in name: return "sm_89"
    except Exception:
        pass
    return "sm_75"
os.environ["GPU_ARCH"] = detect_sm()
if shutil.which("nvcc") is None:
    os.environ["DISABLE_CUDA"] = "1"
    print("nvcc not found → CUDA targets will be skipped.")
else:
    print("nvcc found, arch =", os.environ["GPU_ARCH"])

os.environ["OMP_NUM_THREADS"] = str(min(multiprocessing.cpu_count(), 8))
os.environ["OMP_PROC_BIND"] = "close"
os.environ["OMP_PLACES"] = "cores"

## 4. Build Executables

In [ ]:
!make clean && make all GPU_ARCH=$GPU_ARCH

## 5. Shared Utility Functions

These helper functions are used by the other notebooks for running benchmarks and parsing results. They are included here for completeness but should be copied into each analysis notebook.

In [ ]:
import subprocess, statistics, re, os, json, time, pandas as pd

def run_command(cmd, timeout=300):
    try:
        print("  >", cmd)
        return subprocess.run(cmd, shell=True, capture_output=True,
                             text=True, check=True, timeout=timeout).stdout
    except subprocess.CalledProcessError as e:
        print("    stderr:", e.stderr.strip())
    except subprocess.TimeoutExpired:
        print("    timeout")
    return None

def parse_time(out):
    if not out: return None
    m = re.search(r"time:\s*([0-9]*\.?[0-9]+)\s*(ms|s|sec|seconds)?", out, re.I)
    if not m: return None
    val = float(m.group(1)); unit = (m.group(2) or "s").lower()
    return val/1000.0 if unit.startswith("ms") else val

def time_exe(cmd, warmups=1, runs=3):
    if not cmd: return None
    for _ in range(warmups): _ = run_command(cmd)
    samples = []
    for _ in range(runs):
        t = parse_time(run_command(cmd))
        if t is not None: samples.append(t)
    return statistics.median(samples) if samples else None